# Information Retrieval - Pandemic Investigation  
This notebook retrieves abstracts relevant to pandemics and then uses topic modeling to analyze the chosen abstracts.  Three info retrieval techniques are used: Literal Term Matching, TF-IDF, and Latent Semantic Indexing.  These are linear algebra techniques.  
We use the Scikit-Learn library.

In [5]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import gensim
import time

from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances


#from gensim.models.coherencemodel import CoherenceModel

import TextCleaning

In [6]:
# pull in raw abstracts

raw_df=pd.read_csv('../../data/original/raw_abstracts.csv',engine='python')

# remove null abstracts and duplicates

df = TextCleaning.remove_nulls(raw_df, "ABSTRACT")
df = TextCleaning.remove_duplicates(df)

df.reset_index(inplace = True)
df.rename(columns={'index':'original index'}, inplace=True)

3 nulls in  ABSTRACT . These rows removed.
11 duplicate abstracts removed
0 project ID duplicates - not removed


In [7]:
df.head()

,original index,PROJECT_ID,ABSTRACT,FY,FIRST_CHAR,LAST_CHAR,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_TITLE,PROJECT_TERMS,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,ORGANIZATION_NAME,CFDA_CODE,FY_TOTAL_COST
0,0,89996,"This is a project to explore Game-based, Metap...",2008,"This is a project to explore Game-based, Metap...",.,NSF,NSF,NaN,0814512,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,Achievement; analog; base; Cognitive Science; ...,"REESE, DEBBIE D","CARTER, BEVERLY; WOOD, CHARLES; HITT, BEN",WHEELING JESUIT UNIVERSITY,47.076,1999467.0
1,1,89997,Institution: Franklin Institute Science Museum...,2008,Institution: Franklin Institute Science Museum...,.,NSF,NSF,NaN,0741659,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,Active Learning; Child; Computer software; des...,"SNYDER, STEVEN","ELINICH, KAREN; YOON, SUSAN",FRANKLIN INSTITUTE,47.076,1799699.0
2,2,89998,Through programs (including small group conver...,2008,Through programs (including small group conver...,.,NSF,NSF,NaN,0813522,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,Address; Age; Birth; Brain; Caregivers; Child;...,"FINK, LAURIE KLEINBAUM","CADIGAN, KAREN; ELLENBOGEN, KIRSTEN",SCIENCE MUSEUM OF MINNESOTA,47.076,1505858.0
3,3,89999,In partnership with the American Chemical Soci...,2008,In partnership with the American Chemical Soci...,.,NSF,NSF,NaN,0838627,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,Advanced Development; American; Chemicals; Che...,"JOST, JOHN W","MILLER, BRADLEY; BOWMAN, KATHERINE",INTERNATIONAL UNION OF PURE AND APPLIED CHEMISTRY,47.049,51000.0
4,4,90000,Amphibian populations around the world are exp...,2008,Amphibian populations around the world are exp...,.,NSF,NSF,NaN,0815315,COLLABORATIVE RESEARCH: EVOLUTION OF AMPHIBIAN...,Amphibia; Central America; Communicable Diseas...,"ZAMUDIO, KELLY R",NaN,CORNELL UNIVERSITY ITHACA,47.074,370996.0


In [8]:
# input needed for doc-term matrix creation in Scikit-Learn is one string per document (not a list of strings).  
# Original data is already in this form!

docs = df["ABSTRACT"] 

## Functions needed for all info retrieval approaches

In [9]:
# Create query vector 

def create_query(words, terms):
    
    # words: search query words
    # terms: terms in corpus
    
    q = np.zeros(len(terms))  # number of terms

    idx = []
    for word in query_words:
        idx.append(terms.index(word))

    q[idx] = 1
    
    return q

In [10]:
def return_top_abstracts(docs, scores, top_n):
    
    '''
    docs: Series that contains abstract
    scores: scores of abstracts
    top_n: return the top_n abstracts given by idx
    '''
    # sort scores in descending order
    scores_sorted_idx = np.argsort(scores)[::-1]
    
    ix = scores_sorted_idx[:top_n]
    print(ix[0:10])
    
    return ix, docs[ix]
    

In [11]:
def create_result_df(abstracts, scores):
    
    df = pd.DataFrame()
    df["abstracts"] = abstracts
    df["scores"] = scores
    
    return df

## Literal Term Matching - Frequency Count Document-Term Matrix

This will return all abstracts in the corpus with exact word matches to the query.  

Results will be return in sorted order of how high the query scores with each abstract. A high score means more occurences of the query words in the abstract.

In [12]:
# Create document-term matrix based on count frequencies

#Countvectroizer -> document term matrix, all terms from corpus, add ngrams in this function, contact space tracing?
vectorizer = CountVectorizer()
doc_term_matrix = vectorizer.fit_transform(docs)

In [13]:
terms = vectorizer.get_feature_names()

### Create Query Words - list the search terms

A query is just a list of words to search for in the corpus.  We will use the same query for all three info retrieval techniques.

In [57]:
# CHANGE QUERY WORDS HERE

query_words = ["pandemic"] # other example: ['pandemic', influenza', 'mers', 'sars', 'zikv', 'denv', 'hiv', 'aids']

q = create_query(query_words, terms)

In [15]:
# calculate the score for each document against the query. Docs with more occurences of the query words 
# will score higher

f_scores = doc_term_matrix.dot(q)

In [16]:
sum(f_scores >0)  # how many abstracts include at least one of the query words

16683

In [17]:
# sort scores in descending order

f_scores_sorted = np.sort(f_scores)[::-1]
f_scores_sorted[:10]

array([25., 23., 23., 21., 20., 19., 18., 17., 17., 16.])

In [18]:
f_idx, f_top_abstracts = return_top_abstracts(docs, f_scores, 200)  # CHANGE NUMBER OF TOP DOCS RETURNED

[499832  31654 448561 537114 268530 121339 394637 385407 220044 313343]


In [19]:
f_top_abstracts

499832    Plague is a zoonosis that is present in wild r...
31654     DESCRIPTION (provided by applicant):   Effecti...
448561    ﻿   DESCRIPTION (provided by applicant): Ricke...
537114    Plague is a zoonosis that is present in wild r...
268530    DESCRIPTION (provided by applicant): Men who h...
                                ...                        
125286    DESCRIPTION (provided by applicant): HIV trans...
396178    The studies that assess malaria transmission b...
445087    Studies on asexual stage immunity to P. falcip...
329148    DESCRIPTION (provided by applicant): The virus...
166025    DESCRIPTION (provided by applicant): One of th...
Name: ABSTRACT, Length: 200, dtype: object

In [20]:
f_top_abstracts.iloc[0]

'Plague is a zoonosis that is present in wild rodent populations worldwide and is transmitted primarily by fleas. Yersinia pestis, the plague bacillus, is unique among the enteric group of gram-negative bacteria in having adopted an arthropod-borne route of transmission. Y. pestis has evolved in such a way as to be transmitted during the brief encounter between a feeding flea and a host. A transmissible infection primarily depends on the ability of Y. pestis to grow in the flea as a biofilm that is embedded in a complex extracellular matrix. Bacteria in the biofilm phenotype are deposited into the dermis together with flea saliva, elements which cannot be satisfactorily mimicked by needle-injection of Y. pestis from laboratory cultures. The objective of this project is to identify and determine the function of Y. pestis genes that mediate flea-borne transmission and the initial encounter with the host innate immune system at the infection site in the skin. We study the interaction of Y

In [21]:
f_df = create_result_df(docs, f_scores)

In [22]:
f_df

,abstracts,scores
0,"This is a project to explore Game-based, Metap...",0.0
1,Institution: Franklin Institute Science Museum...,0.0
2,Through programs (including small group conver...,0.0
3,In partnership with the American Chemical Soci...,0.0
4,Amphibian populations around the world are exp...,0.0
...,...,...
550069,The Title IV-E Prevention Services Clearinghou...,0.0
550070,This mixed-methods study seeks to deepen our u...,0.0
550071,The purpose of this project is to examine the ...,0.0
550072,The 2014 Child Care and Development Block Gran...,0.0


## TF-IDF Document-Term Matrix

This approach is similar to Literal Term Matching using frequency counts in the document-term matrix.  However, instead of using frequency counts, the entries of the document-term matrix are weighted using TF-IDF.

In [23]:
# Find doc-term matrix using TF-IDF weighting
#Added this line
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer = TfidfVectorizer()
tf_idf = tf_idf_vectorizer.fit_transform(docs)

In [24]:
tf_idf_terms = tf_idf_vectorizer.get_feature_names()  # these terms are the same as the terms created from the 
                                                      # frequency count document-term matrix, so we do not need to
                                                      # recreate the query vector.

In [25]:
# calculate the score for each document against the query. Docs with more occurences of the query words 
# will score higher

tf_idf_scores = tf_idf.dot(q)

In [26]:
sum(tf_idf_scores >0)   # how many abstracts include at least one of the query words

16683

In [27]:
# sort scores in descending order

tf_idf_scores_sorted = np.sort(tf_idf_scores)[::-1]
tf_idf_scores_sorted[:10]

array([0.56967192, 0.52730382, 0.49964416, 0.48303602, 0.47775469,
       0.47298193, 0.46335521, 0.45578613, 0.45450377, 0.44761227])

In [28]:
tfidf_idx, tfidf_top_abstracts = return_top_abstracts(docs, tf_idf_scores, 200)  # CHANGE NUMBER OF TOP DOCS RETURNED

[445075 268530 376797 313343  31654 385407 374280 419188 549728 435116]


In [29]:
tfidf_top_abstracts

445075    This research will examine the epidemiological...
268530    DESCRIPTION (provided by applicant): Men who h...
376797    DESCRIPTION (provided by applicant): There has...
313343    DESCRIPTION (provided by applicant): Men who h...
31654     DESCRIPTION (provided by applicant):   Effecti...
                                ...                        
548042    Energy systems are transitioning from fossil f...
403436    ﻿   DESCRIPTION (provided by applicant Adolesc...
386857    Social interactions cause the spread of parasi...
240085    DESCRIPTION (provided by applicant): HIV conti...
178353    DESCRIPTION (provided by applicant): HIV conti...
Name: ABSTRACT, Length: 200, dtype: object

In [30]:
tf_idf_df = create_result_df(docs, tf_idf_scores)

In [31]:
tf_idf_df

,abstracts,scores
0,"This is a project to explore Game-based, Metap...",0.0
1,Institution: Franklin Institute Science Museum...,0.0
2,Through programs (including small group conver...,0.0
3,In partnership with the American Chemical Soci...,0.0
4,Amphibian populations around the world are exp...,0.0
...,...,...
550069,The Title IV-E Prevention Services Clearinghou...,0.0
550070,This mixed-methods study seeks to deepen our u...,0.0
550071,The purpose of this project is to examine the ...,0.0
550072,The 2014 Child Care and Development Block Gran...,0.0


## Latent Semantic Indexing (LSI) Approach

LSI Uses the TF-IDF matrix.  LSI is a tecnique that utilizes a truncated Singular Value Decomposition of the document-term matrix.  Basically, LSI still returns relevant documents to the query; however some of the documents returned may not include the exact search terms!  LSI is finding the latent or hidden relationships in the terms.  

In [32]:
# Find the Truncated SVD of the TF-IDF matrix

lsa = TruncatedSVD(n_components=500, random_state=1)  # CHANGE THE NUMBER OF COMPONENTS - NOTE: MORE COMPONENTS 
                                                      # GIVES YOU A MORE ACCURATE APPROXIMATION OF THE DOC-TERM 
                                                      # MATRIX, BUT IS ALSO MORE EXPENSIVE AND MAY NOT LEAD TO THE 
                                                      # BEST INFO RETRIEVAL RESULTS.
USigma = lsa.fit_transform(tf_idf)
Vtrans = lsa.components_

In [33]:
# transform query to be in same space as documents

q = q.reshape(1,-1)
qhat = lsa.transform(q)

In [34]:
print(qhat.shape)
print(USigma.shape)
print(Vtrans.shape)

(1, 500)
(550074, 500)
(500, 1058314)


In [35]:
lsa_scores = pairwise_distances(qhat, USigma, metric='cosine', n_jobs=7)  # CHANGE N_JOBS TO BE NUMBER OF CORES - 1

In [36]:
lsa_scores.shape

(1, 550074)

In [37]:
lsa_scores

array([[1.00870104, 1.02560024, 0.99659785, ..., 0.96097061, 0.98673479,
        0.95503352]])

In [38]:
type(lsa_scores)

numpy.ndarray

In [39]:
sum(lsa_scores[0] > 0)  # how many abstracts scored above 0

550074

In [40]:
lsa_scores[0]

array([1.00870104, 1.02560024, 0.99659785, ..., 0.96097061, 0.98673479,
       0.95503352])

In [41]:
# sort scores in descending order

lsa_scores_sorted = np.sort(lsa_scores[0])[::-1]
lsa_scores_sorted[:10]

array([1.18624546, 1.17515152, 1.17442921, 1.1737514 , 1.17311561,
       1.17080677, 1.17071588, 1.16935998, 1.16270814, 1.16019365])

In [42]:
lsa_idx, lsa_top_abstracts = return_top_abstracts(docs, lsa_scores[0], 200)  # CHANGE NUMBER OF TOP DOCS RETURNED

[497971 176653 531263 414783  18799 483108 536266 410792 483850 186024]


In [43]:
lsa_top_abstracts

497971    The major goal of this program is to further d...
176653    DESCRIPTION (provided by applicant): Our long ...
531263    Project SummaryRespiratory syncytial virus (RS...
414783    ﻿   DESCRIPTION (provided by applicant): Respi...
18799     DESCRIPTION (provided by applicant): Respirato...
                                ...                        
212525    DESCRIPTION (provided by applicant): The hepat...
251661    DESCRIPTION (provided by applicant): The hepat...
494241    PROJECT ABSTRACT Retroviruses are single-stran...
217288    DESCRIPTION (provided by applicant: Respirator...
352693    RSV and HMPV are cytoplasmic enveloped RNA vir...
Name: ABSTRACT, Length: 200, dtype: object

In [44]:
lsa_df = create_result_df(docs, lsa_scores[0])

In [45]:
lsa_df

,abstracts,scores
0,"This is a project to explore Game-based, Metap...",1.008701
1,Institution: Franklin Institute Science Museum...,1.025600
2,Through programs (including small group conver...,0.996598
3,In partnership with the American Chemical Soci...,1.014584
4,Amphibian populations around the world are exp...,0.948461
...,...,...
550069,The Title IV-E Prevention Services Clearinghou...,1.013772
550070,This mixed-methods study seeks to deepen our u...,0.954471
550071,The purpose of this project is to examine the ...,0.960971
550072,The 2014 Child Care and Development Block Gran...,0.986735


## Create pandemics corpus

We use the results of our three information retrieval techniques to create a new, smaller corpus that only contains abstracts relevant to the query. 

In [46]:
docs_ix = np.concatenate([f_idx, tfidf_idx, lsa_idx])

In [47]:
docs_idx = np.unique(docs_ix)

In [48]:
docs_idx.shape

(487,)

In [49]:
lim_docs = [docs[i] for i in docs_idx]

In [50]:
# print out the abstracts of the pandemics corpus -- long text output!

lim_docs

['Cooperative transmission is an emerging wireless communication paradigm that improves wireless channel capacity by creating multi-user cooperation in the physical layer. In cooperative transmission, when the source node transmits a message to the destination node, the nearby nodes that overheard this transmission will  help  the source and destination by relaying the replicas of the message, and the destination will combine the multiple received waveforms so as to improve the link quality. This technology departs from the traditional point-to-point link abstraction of wireless connections, and is expected to have a profound impact on network performance and design. The current algorithm design, protocol development and performance evaluation for cooperative transmission assume that all wireless nodes are trustworthy. Investigation of the security issues is lagging behind. This research addresses the major security issues related to cooperative transmission. In particular, does cooper

## Topic Modeling with relevant pandemic abstracts

In [51]:
# Create a TF-IDF document-term matrix for the pandemics corpus 

# TRY DIFFERENT PARAMETERS IN THE TF-IDF DOC-TERM MATRIX SET-UP
nmf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2) , #tfdi vector see for contact, run 1-2, bigrams - 2 word phrases occured right after the other as one token, if we had n_gram range from1-3, unigrams - 1 word token, bigrams and trigrams
                                 max_df=0.4, min_df=3, lowercase=True) #, max_features=int(len(lim_docs)/2))

nmf_tf_idf = nmf_vectorizer.fit_transform(lim_docs)

In [52]:
# topic modeling with NMF

nmf_model = NMF(n_components=10, random_state=1)  # TRY DIFFERENT NUMBERS OF TOPICS
W = nmf_model.fit_transform(nmf_tf_idf)
H = nmf_model.components_

In [53]:
# TRY TOPIC MODELING WITH LDA



In [54]:
# function slightly modified from https://nlpforhackers.io/topic-modeling/

def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):  # loop through each row of H.  idx = row index.  topic = actual row
        print("\nTopic %d:" % (idx))
        #print([(vectorizer.get_feature_names()[i], topic[i])  # printing out words corresponding to indices found in next line
                        #for i in topic.argsort()[:-top_n - 1:-1]])  # finding indices of top words in topic
            
        print_list = [(vectorizer.get_feature_names()[i], topic[i])  
                        for i in topic.argsort()[:-top_n - 1:-1]]
        for item in print_list:
            print(item)
        

In [55]:
print_topics(nmf_model, nmf_vectorizer, 10)


Topic 0:
('rsv', 1.5934007699962716)
('vaccine', 0.7800857798694228)
('respiratory', 0.2800248304788406)
('protein', 0.2691293390856946)
('vaccines', 0.23607012197907648)
('attenuated', 0.2264763164038899)
('antibodies', 0.20853878416478813)
('responses', 0.2028429840736325)
('candidates', 0.20025845860555877)
('antibody', 0.19479227176425193)

Topic 1:
('hiv', 1.6349380023774915)
('prevention', 0.23637857337692092)
('sexual', 0.19428726459744267)
('cell', 0.19110771925292516)
('risk', 0.1665908100113924)
('networks', 0.14794296397228338)
('clinical', 0.14650858161023708)
('core', 0.1460775884696191)
('partners', 0.14201944061245733)
('microbicides', 0.14011624536443226)

Topic 2:
('malaria', 1.2280788137139287)
('parasite', 0.49031114852172886)
('mosquito', 0.39998636765979595)
('blocking', 0.39657948458181785)
('mosquitoes', 0.3065048470553618)
('falciparum', 0.29183697282089144)
('assay', 0.20617637895103638)
('smfa', 0.20253424191986985)
('parasites', 0.20127841788331632)
('plasmo